# Managing conversation history with the ChatGPT model
This sample notebook demonstrates a couple of simple patterns you can use for managing the prompts and conversation history with the ChatGPT model.

In [23]:
# if needed, install and/or upgrade to the latest version of the OpenAI Python library
%pip install --upgrade openai

Note: you may need to restart the kernel to use updated packages.


In [27]:
# import os module & the OpenAI Python library for calling the OpenAI API
# please make sure you have installed required libraries via pip install -r requirements.txt
import os, dotenv
import openai

dotenv.load_dotenv()

True

In [28]:
# Setting up the deployment name
deployment_name = os.environ['CHAT_COMPLETIONS_MODEL']

# The base URL for your Azure OpenAI resource. e.g. "https://<your resource name>.openai.azure.com"
azure_endpoint = os.environ['AZURE_OPENAI_ENDPOINT']

# The API key for your Azure OpenAI resource.
api_key = os.environ['AZURE_OPENAI_API_KEY']

# Currently OPENAI API have the following versions available: https://learn.microsoft.com/azure/ai-services/openai/reference
api_version = os.environ['OPENAI_API_VERSION']

client = openai.AzureOpenAI(
  azure_endpoint = azure_endpoint, 
  api_key=api_key,  
  api_version=api_version
)

## 1.0 Create the system message for ChatGPT

In [29]:
base_system_message = "You are a helpful assistant."

system_message = f"{base_system_message.strip()}"
print(system_message)

You are a helpful assistant.


## 2.0 Define helper functions



In [30]:
import tiktoken

def num_tokens_from_messages(messages, model="gpt-3.5-turbo-0301"):
    encoding = tiktoken.encoding_for_model(model)
    num_tokens = 0
    for message in messages:
        num_tokens += 4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":  # if there's a name, the role is omitted
                num_tokens += -1  # role is always required and always 1 token
    num_tokens += 2  # every reply is primed with <im_start>assistant
    return num_tokens

In [31]:
# Defining a function to send the prompt to the ChatGPT model
# More info : https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/chatgpt?pivots=programming-language-chat-completions
def send_message(messages, model_name, max_response_tokens=500):
    response = client.chat.completions.create(
        model=model_name,
        messages=messages,
        temperature=0.5,
        max_tokens=max_response_tokens,
        top_p=0.9,
        frequency_penalty=0,
        presence_penalty=0,
    )
    return response.choices[0].message.content

# Defining a function to print out the conversation in a readable format
def print_conversation(messages):
    for message in messages:
        print(f"[{message['role'].upper()}]")
        print(message['content'])
        print()

## 3.0 Start the conversation

In [32]:
# This is the first user message that will be sent to the model. Feel free to update this.
user_message = "I want to write a blog post about the impact of AI on the future of work."

In [33]:
# Create the list of messages. role can be either "user" or "assistant" 
messages=[
    {"role": "system", "content": system_message},
    {"role": "user", "name":"example_user", "content": user_message}
]

In [34]:
token_count = num_tokens_from_messages(messages)
print(f"Token count: {token_count}")

Token count: 37


In [35]:
max_response_tokens = 500

response = send_message(messages, deployment_name, max_response_tokens)
messages.append({"role": "assistant", "content": response})

print_conversation(messages)

[SYSTEM]
You are a helpful assistant.

[USER]
I want to write a blog post about the impact of AI on the future of work.

[ASSISTANT]
Great topic choice! The impact of AI on the future of work is indeed a fascinating and relevant subject. Here's a structure you can follow to write your blog post:

1. Introduction:
   - Start with a captivating opening statement or a thought-provoking question to grab the reader's attention.
   - Provide a brief overview of AI and its growing significance in various industries.

2. The current state of AI in the workplace:
   - Discuss how AI is already being used in different sectors, such as healthcare, finance, manufacturing, and customer service.
   - Highlight some specific examples of AI technologies and their applications.

3. Advantages of AI in the workplace:
   - Explain the potential benefits that AI brings to businesses and employees.
   - Discuss how AI can automate repetitive tasks, enhance productivity, and improve decision-making processe

## 4.0 Continue the conversation

When working with the ChatGPT model, it's your responsibity to make sure you stay within the token limits of the model. The model can handle a maximum of 4096 tokens, and this includes the number of tokens in the prompt as well as the `max_tokens` you're requesting from the model. If you exceed these limits, the model will return an error.

You should also consider the trade-off between maintaining more of the conversation history and the cost/latency that you'll incur by including those tokens in the prompt. Shorter prompts are cheaper and faster. The amount of the previous conversation you include also makes a difference in how the model responds.

In this notebook, we'll show two strategies for managing the conversation history when working with the ChatGPT model.
- Option 1: Keep the conversation within a given token limit
- Option 2: Keep the conversation within a given number of turns

### Option 1: Keep the conversation within a given token limit

`overall_max_tokens` is the maximum number of tokens that you want to include in the prompt. Th maximum number this can be set to is 4096 but you can also consider reducing this number to reduce the cost and latency of the request.

In [36]:
overall_max_tokens = 4096
prompt_max_tokens = overall_max_tokens - max_response_tokens

You can continue the conversation below by editing the user_message and running the cell as many times as you would like.

In [38]:
user_message = "The target audience for the blog post should be business leaders working in the tech industry."
#user_message = "Let's talk about generative AI and keep the tone informational but also friendly."
#user_message = "Show me a few more examples"
messages.append({"role": "user", "content": user_message})

token_count = num_tokens_from_messages(messages)
print(f"Token count: {token_count}")

# remove first message while over the token limit
while token_count > prompt_max_tokens:
    messages.pop(0)
    token_count = num_tokens_from_messages(messages)

response = send_message(messages, deployment_name, max_response_tokens)

messages.append({"role": "assistant", "content": response})
print_conversation(messages)

Token count: 865
[SYSTEM]
You are a helpful assistant.

[USER]
I want to write a blog post about the impact of AI on the future of work.

[ASSISTANT]
Great topic choice! The impact of AI on the future of work is indeed a fascinating and relevant subject. Here's a structure you can follow to write your blog post:

1. Introduction:
   - Start with a captivating opening statement or a thought-provoking question to grab the reader's attention.
   - Provide a brief overview of AI and its growing significance in various industries.

2. The current state of AI in the workplace:
   - Discuss how AI is already being used in different sectors, such as healthcare, finance, manufacturing, and customer service.
   - Highlight some specific examples of AI technologies and their applications.

3. Advantages of AI in the workplace:
   - Explain the potential benefits that AI brings to businesses and employees.
   - Discuss how AI can automate repetitive tasks, enhance productivity, and improve decisio

### Option 2: Keep the conversation within a given number of turns

In [39]:
max_messages = 10

overall_max_tokens = 4096
prompt_max_tokens = overall_max_tokens - max_response_tokens

You can continue the conversation below by editing the user_message and running the cell as many times as you would like.

In [41]:
user_message = "Make the post about generative AI aimed at business leaders who have some knowledge of the technology."
messages.append({"role": "user", "content": user_message})

while len(messages) > max_messages:
    messages.pop(0)

token_count = num_tokens_from_messages(messages)

while token_count > prompt_max_tokens:
    # remove first message from messages
    messages.pop(0)
    token_count = num_tokens_from_messages(messages)

response = send_message(messages, deployment_name)
messages.append({"role": "assistant", "content": response})
print_conversation(messages)


[SYSTEM]
You are a helpful assistant.

[USER]
I want to write a blog post about the impact of AI on the future of work.

[ASSISTANT]
Great topic choice! The impact of AI on the future of work is indeed a fascinating and relevant subject. Here's a structure you can follow to write your blog post:

1. Introduction:
   - Start with a captivating opening statement or a thought-provoking question to grab the reader's attention.
   - Provide a brief overview of AI and its growing significance in various industries.

2. The current state of AI in the workplace:
   - Discuss how AI is already being used in different sectors, such as healthcare, finance, manufacturing, and customer service.
   - Highlight some specific examples of AI technologies and their applications.

3. Advantages of AI in the workplace:
   - Explain the potential benefits that AI brings to businesses and employees.
   - Discuss how AI can automate repetitive tasks, enhance productivity, and improve decision-making processe